In [5]:
import os
GPU_id = 1
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_id)

from torch_scatter import *
from torch_geometric.utils import scatter_
import torch
import torch.nn as nn
import numbers

import torch
from torch import _utils
from fastai.torch_core import to_device
import torch.nn.functional as F 

from fastai.basic_data import DataBunch
from fastai.basic_data import *
from fastai.tabular import *
from fastai import *

import copy
import warnings
warnings.filterwarnings("ignore") 

from mpnn_model.common import *
from mpnn_model.helpers import load_cfg
from mpnn_model.model import *
from mpnn_model.common_constants import * 

DATA_DIR = '/rapids/notebooks/srabhi/champs-2019/input'


In [6]:
model_dict = { '1JHC': 'lmae', '2JHC': 'lmae', '3JHC': 'lmae', '3JHH': 'lmae',
             '1JHN': 'mlmae' , '2JHN':'mlmae' , '3JHN':'mlmae', '2JHH':'mlmae'}

In [7]:
# to modify with the resulting model of self_training 
best_all_type_model_lmae = './models/mpnn_gauss_rank_LMAE_2CE_RNN_NODE_TYPE_ATOMIC_SEQ_BOOTSTRAP_V2_'
best_all_type_model_mlmae = './models/mpnn_gauss_rank_MLMAE_2CE_RNN_NODE_TYPE_ATOMIC_SEQ_BOOTSTRAP_V2_'

### Without GAUSSRANK : 

In [10]:
def save_model(fold, coupling_type, loss_criterion='lmae'): 
    cfg = load_cfg('./experiments/MPNN_RNN_MAE_WO_GAUSSRANK_SINGLE_TYPE.yaml')
    if loss_criterion == 'lmae': 
        pretrained_dict =  torch.load(best_all_type_model_lmae+'_fold_%s_final_save.pth'%fold).state_dict()
    else:
        pretrained_dict =  torch.load(best_all_type_model_lmae+'_fold_%s_final_save.pth'%fold).state_dict()
    ind = COUPLING_TYPE.index(coupling_type)
    net= Net(cfg, y_range=[COUPLING_MIN_[ind], COUPLING_MAX_[ind]])
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and not k.startswith('predict')}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict) 
    # 3. load the new state dict
    net.load_state_dict(pretrained_dict, strict=False)
    # save model 
    torch.save(net, 'pre_trained_models/' + 'coupling_%s_%s_fold_%s_wo_gaussrank.pth'%(coupling_type, loss_criterion, fold))

In [11]:
for fold in range(4):
    for type_, criterion in model_dict.items():
        save_model(fold, type_, criterion)

### With Gaussrank 

In [12]:
def save_model(fold, coupling_type, loss_criterion='lmae'): 
    cfg = load_cfg('/rapids/notebooks/srabhi/champs-2019/CherKeng_solution/fastai_code/experiments/MPNN_RNN_MAE_GAUSSRANK_SINGLE_TYPE.yaml')
    if loss_criterion == 'lmae': 
        pretrained_dict =  torch.load('models/mpnn_gauss_rank_LMAE_2CE_RNN_NODE_TYPE_ATOMIC_SEQ_BOOTSTRAP_V2__fold_%s_final_save.pth'%fold).state_dict()
    else:
        pretrained_dict =  torch.load('models/mpnn_gauss_rank_MLMAE_2CE_RNN_NODE_TYPE_ATOMIC_SEQ_BOOTSTRAP_V2__fold_%s_final_save.pth'%fold).state_dict()
    ind = COUPLING_TYPE.index(coupling_type)
    net= Net(cfg, y_range=[-2.326753765513524, 2.3267537655135464])
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and not k.startswith('predict')}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict) 
    # 3. load the new state dict
    net.load_state_dict(pretrained_dict, strict=False)
    # save model 
    torch.save(net, 'pre_trained_models/' + 'coupling_%s_%s_fold_%s_gaussrank.pth'%(coupling_type, loss_criterion, fold))

In [13]:
for fold in range(4):
    for type_, criterion in model_dict.items():
        save_model(fold, type_, criterion)